In [451]:
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans

In [452]:
df=pd.read_csv('flipkart_com-ecommerce_sample.csv',index_col=0)
df=df.dropna()
df=df.drop(['crawl_timestamp','product_url','overall_rating'],axis=1)
df=df.reset_index()

In [453]:
a=np.array(df['product_rating'])
a=[float(x) if x!="No rating available" else 4.1 for x in a]

In [454]:
df['product_rating_mediated']=a

In [455]:
category_tree=list(df['product_category_tree'])
category_tree=[x[2:-2] for x in category_tree]
split_category_tree=[x.split(' >>') for x in category_tree]
category=[]
for x in split_category_tree:
    category.append(x[0])
sub_category=[]
for x in split_category_tree:
    if(len(x)>1):
        sub_category.append(x[1])
    else:
        sub_category.append(x[0])


In [456]:
df['category']=category
df['sub_category']=sub_category

In [457]:
df

,uniq_id,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,brand,product_specifications,product_rating_mediated,category,sub_category
0,c2d766ca982eca8304150849735ffef9,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",4.1,Clothing,Women's Clothing
1,7f7036a6d550aaa89d34c77bd39a5e48,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati...",4.1,Furniture,Living Room Furniture
2,f449ec65dcbc041b6ae5e6a32717d01b,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""...",4.1,Footwear,Women's Footwear
3,0973b37acd0c664e3de26e97e5571454,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",4.1,Clothing,Women's Clothing
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",...",4.1,Pet Supplies,Grooming
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14053,7179d2f6c4ad50a17d014ca1d2815156,WallDesign Small Vinyl Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",STIE7KFJAKSTDY9G,1500.0,730.0,"[""http://img6a.flixcart.com/image/wall-decorat...",False,Buy WallDesign Small Vinyl Sticker for Rs.730 ...,No rating available,WallDesign,"{""product_specification""=>[{""key""=>""Number of ...",4.1,Baby Care,Baby & Kids Gifts
14054,71ac419198359d37b8fe5e3fffdfee09,Wallmantra Large Vinyl Stickers Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",STIE9F5URNQGJCGH,1429.0,1143.0,"[""http://img6a.flixcart.com/image/sticker/z/g/...",False,Buy Wallmantra Large Vinyl Stickers Sticker fo...,No rating available,Wallmantra,"{""product_specification""=>[{""key""=>""Number of ...",4.1,Baby Care,Baby & Kids Gifts
14055,93e9d343837400ce0d7980874ece471c,Elite Collection Medium Acrylic Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",STIE7VAYDKQZEBSD,1299.0,999.0,"[""http://img5a.flixcart.com/image/sticker/b/s/...",False,Buy Elite Collection Medium Acrylic Sticker fo...,No rating available,Elite Collection,"{""product_specification""=>[{""key""=>""Number of ...",4.1,Baby Care,Baby & Kids Gifts
14056,669e79b8fa5d9ae020841c0c97d5e935,Elite Collection Medium Acrylic Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",STIE8YSVEPPCZ42Y,1499.0,1199.0,"[""http://img5a.flixcart.com/image/sticker/4/2/...",False,Buy Elite Collection Medium Acrylic Sticker fo...,No rating available,Elite Collection,"{""product_specification""=>[{""key""=>""Number of ...",4.1,Baby Care,Baby & Kids Gifts


In [458]:
def random_float_in_range(mean, std_dev, min_val, max_val):
    while True:
        value = np.random.normal(mean, std_dev)
        if min_val <= value <= max_val:
            return round(value,1)

min_range = 1
max_range = 5
mean_value = (min_range + max_range) / 2
std_deviation = (max_range - min_range) / 6  

random_float = random_float_in_range(mean_value, std_deviation, min_range, max_range)

In [459]:
import random
item_id=[]
ratings=[]
num_of_cust=1000
num_of_products=100
for x in range(num_of_cust):
    temp=[]
    temp2=[]
    for y in range(num_of_products):
        temp.append(random.randint(0,len(df)-1))
        temp2.append(random_float_in_range(mean_value, std_deviation, min_range, max_range))
    item_id.append(temp)
    ratings.append(temp2)

In [460]:
customers=pd.DataFrame()
customers['item_id']=item_id
customers['ratings']=ratings

In [461]:
customers

,item_id,ratings
0,"[458, 4196, 9504, 4243, 4357, 1459, 13753, 338...","[2.8, 4.1, 1.7, 3.1, 3.6, 2.5, 2.8, 3.8, 2.8, ..."
1,"[10554, 13995, 3809, 10002, 6471, 4574, 3816, ...","[3.2, 3.8, 3.9, 1.8, 1.7, 2.3, 2.7, 3.1, 3.1, ..."
2,"[6130, 11538, 7068, 7700, 8526, 7048, 13427, 9...","[1.7, 3.5, 2.3, 3.8, 1.9, 3.7, 2.1, 3.4, 3.7, ..."
3,"[2616, 889, 312, 11639, 177, 7964, 2093, 2818,...","[3.9, 2.0, 2.4, 3.8, 2.5, 2.5, 2.0, 2.4, 4.0, ..."
4,"[12630, 2824, 7294, 3455, 9599, 7363, 6432, 17...","[2.3, 3.2, 2.4, 3.5, 2.5, 3.9, 1.9, 2.0, 2.9, ..."
...,...,...
995,"[11427, 8033, 10760, 10805, 6417, 12761, 8326,...","[2.9, 3.4, 3.4, 2.3, 2.8, 2.4, 3.4, 1.7, 3.5, ..."
996,"[1329, 12349, 4197, 7866, 11198, 5007, 794, 66...","[3.1, 3.2, 2.5, 2.8, 2.3, 2.9, 2.8, 2.6, 3.6, ..."
997,"[310, 10113, 4549, 11205, 10118, 412, 8127, 64...","[1.6, 2.0, 4.1, 3.9, 2.2, 3.1, 2.6, 3.7, 2.6, ..."
998,"[11290, 10684, 11464, 11318, 2086, 918, 10977,...","[3.4, 4.3, 2.4, 2.8, 3.2, 3.0, 2.3, 2.9, 3.7, ..."


In [462]:
label_encoder = LabelEncoder()
encoded_categories = label_encoder.fit_transform(df['category'])
df['encoded_categories']=encoded_categories

In [463]:
num_rows = num_of_cust
num_columns = len(df['category'].unique())
data = [[0] * num_columns for _ in range(num_rows)]
columns = [i for i in range(num_columns)]

cust_encoded = pd.DataFrame(data, columns=columns)
cust_encoded

,0,1,2,3,4,5,6,7,8,9,...,251,252,253,254,255,256,257,258,259,260
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [464]:
df.iloc[256]

uniq_id                                     9d9bd8488bf09dc468ecf58f7caeeaf0
product_name                        Karatcraft Trilic Gold Diamond 18 K Ring
product_category_tree                                 ["Jewellery >> Rings"]
pid                                                         RNGE88FRA9DMA4UZ
retail_price                                                         49999.0
discounted_price                                                     33761.0
image                      ["http://img5a.flixcart.com/image/ring/y/h/x/r...
is_FK_Advantage_product                                                False
description                Karatcraft Trilic Gold Diamond 18 K Ring\n    ...
product_rating                                           No rating available
brand                                                             Karatcraft
product_specifications     {"product_specification"=>[{"key"=>"Pack of", ...
product_rating_mediated                                                  4.1

In [465]:
for i in range(len(customers)):
    cust=customers.iloc[i]
    for j in range(len(cust['item_id'])):
        # print(i,j)
        # print(cust['item_id'][j])
        # print(df.iloc[cust['item_id'][j]]['encoded_categories'])
        col=df.iloc[cust['item_id'][j]]['encoded_categories']
        val=cust['ratings'][j]
        cust_encoded.iloc[i][col]+=val
        # print(cust_encoded.iloc[i][col])

print(cust_encoded.value_counts())

0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99  100  101  102  103  104  105  106  107  108  109  110  111  112  113  114  115  116  117  118  119  120  121  122  123  124  125  126  127  128  129  130  131  132  133  134  135  136  137  138  139  140  141  142  143  144  145  146  147  148  149  150  151  152  153  154  155  156  157  158  159  160  161  162  163  164  165  166  167  168  169  170  171  172  173  174  175  176  177  178  179  180  181  182  183  184  185  186  187  188  189  190  191  192  193  194  195  196  197  198  199  200  201  202  203  204  205  206  207  208  209  210  211  212  213  214  215  216  217  218  219  220  221  

In [466]:
num_clusters = int(len(cust_encoded)/10)
kmeans = KMeans(n_clusters=num_clusters)
cluster_labels = kmeans.fit_predict(cust_encoded)

# Add cluster labels to the DataFrame
cust_encoded['cluster_label']=cluster_labels
# cust_encoded

c:\Users\janma\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [467]:
cust_encoded['cluster_label'].value_counts()

97    27
60    26
30    25
2     24
26    22
      ..
89     2
84     1
31     1
56     1
55     1
Name: cluster_label, Length: 100, dtype: int64

In [468]:
median_value = np.median(cluster_labels)

index_closest_to_median = np.argmin(np.abs(cluster_labels - median_value))

# Get the label (value) corresponding to the index
label_of_median = cluster_labels[index_closest_to_median]


In [469]:
label=np.bincount(cluster_labels).argmax()
group=cust_encoded[cust_encoded['cluster_label']==label]
target_customer=group.index[0]
group

,0,1,2,3,4,5,6,7,8,9,...,252,253,254,255,256,257,258,259,260,cluster_label
13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,97
35,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,97
111,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,97
171,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,97
185,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,97
295,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,97
319,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,97
336,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,97
388,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,97
420,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,97


In [470]:
len(df['sub_category'].unique())

442

In [471]:
label_encoder = LabelEncoder()
encoded_subcategories = label_encoder.fit_transform(df['sub_category'])
df['encoded_subcategories']=encoded_subcategories

In [472]:
expanded_group=pd.DataFrame()
num_rows = cust_encoded['cluster_label'].value_counts()[label]
num_columns = len(df['sub_category'].unique())
data = [[0] * num_columns for _ in range(num_rows)]
columns = [i for i in range(num_columns)]

expanded_group = pd.DataFrame(data, columns=columns)
expanded_group.set_index(pd.Index(group.index),inplace=True)


In [473]:
for i in range(len(expanded_group)):
    cust_index=expanded_group.index[i]
    cust=customers.iloc[cust_index]
    for j in range(len(cust['item_id'])):
        col=df.iloc[cust['item_id'][j]]['encoded_subcategories']
        val=cust['ratings'][j]
        expanded_group.iloc[i][col]+=val
expanded_group_t=(expanded_group.transpose())
expanded_group_t

,13,35,111,171,185,295,319,336,388,420,...,622,671,694,710,726,730,751,861,893,982
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2,0,0,0,0,0,3,3,0,0,...,0,0,0,0,0,0,0,0,3,0
4,0,0,6,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
438,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
439,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
440,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [474]:
row_sums = expanded_group_t.sum(axis=1)
expanded_group_t['sum_of_ratings']=row_sums
expanded_group_t=expanded_group_t[expanded_group_t['sum_of_ratings']!=0]
expanded_group_t

,13,35,111,171,185,295,319,336,388,420,...,671,694,710,726,730,751,861,893,982,sum_of_ratings
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
3,2,0,0,0,0,0,3,3,0,0,...,0,0,0,0,0,0,0,3,0,11
4,0,0,6,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6
10,0,0,0,0,0,0,0,0,4,0,...,0,0,0,0,0,0,0,0,0,4
12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
16,0,0,3,3,0,3,3,0,0,0,...,0,2,2,4,0,0,0,3,3,32
19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,2
22,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
23,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
37,0,0,0,3,0,0,0,2,2,0,...,0,0,0,0,0,0,0,0,0,7


In [475]:
expanded_group_t=expanded_group_t.drop('sum_of_ratings',axis=1)

In [476]:
num_clusters = int(len(expanded_group_t)/10)
kmeans = KMeans(n_clusters=num_clusters)
cluster_labels = kmeans.fit_predict(expanded_group_t)

# Add cluster labels to the DataFrame
expanded_group_t['cluster_label']=cluster_labels

c:\Users\janma\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [477]:
expanded_group_t

,13,35,111,171,185,295,319,336,388,420,...,671,694,710,726,730,751,861,893,982,cluster_label
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,2,0,0,0,0,0,3,3,0,0,...,0,0,0,0,0,0,0,3,0,4
4,0,0,6,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
10,0,0,0,0,0,0,0,0,4,0,...,0,0,0,0,0,0,0,0,0,1
12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
16,0,0,3,3,0,3,3,0,0,0,...,0,2,2,4,0,0,0,3,3,4
19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,1
22,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
23,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
37,0,0,0,3,0,0,0,2,2,0,...,0,0,0,0,0,0,0,0,0,1


In [478]:
expanded_group_t['cluster_label'].value_counts()

1    49
4     5
0     2
2     1
3     1
Name: cluster_label, dtype: int64

In [479]:
target_customer

13

In [480]:
ind=customers.iloc[target_customer]['item_id'][-1]
row=df.iloc[ind]['encoded_subcategories']

In [481]:
target_cluster_label=expanded_group_t.loc[row]['cluster_label']

In [482]:
req_cust=pd.DataFrame(expanded_group_t[expanded_group_t['cluster_label']==target_cluster_label])
req_cust.drop('cluster_label',axis=1,inplace=True)
req_cust

,13,35,111,171,185,295,319,336,388,420,...,622,671,694,710,726,730,751,861,893,982
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,6,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,4,0,...,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
22,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
37,0,0,0,3,0,0,0,2,2,0,...,0,0,0,0,0,0,0,0,0,0
39,0,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,0
44,0,0,0,0,0,0,0,0,0,0,...,3,0,0,0,0,0,0,0,0,0


In [483]:
row_sums = req_cust.sum(axis=1)
req_cust['sum_of_ratings']=row_sums
req_cust=req_cust[req_cust['sum_of_ratings']>2]
req_cust.drop('sum_of_ratings',axis=1,inplace=True)

C:\Users\janma\AppData\Local\Temp\ipykernel_17728\4239025566.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_cust.drop('sum_of_ratings',axis=1,inplace=True)


In [484]:
num_clusters = int((len(req_cust)+10)/10)
kmeans = KMeans(n_clusters=num_clusters)
cluster_labels = kmeans.fit_predict(req_cust)

# Add cluster labels to the DataFrame
req_cust['cluster_label']=cluster_labels

c:\Users\janma\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\janma\AppData\Local\Temp\ipykernel_17728\2178137360.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_cust['cluster_label']=cluster_labels


In [485]:
req_cust['cluster_label'].value_counts()

1    24
4     5
0     4
2     4
3     3
Name: cluster_label, dtype: int64

In [486]:
ind=customers.iloc[target_customer]['item_id'][-1]
row=df.iloc[ind]['encoded_subcategories']
target_cluster_label=req_cust.loc[row]['cluster_label']
target_cluster_label

4

In [488]:
final_cust=pd.DataFrame(req_cust[req_cust['cluster_label']==target_cluster_label])
final_cust

,13,35,111,171,185,295,319,336,388,420,...,671,694,710,726,730,751,861,893,982,cluster_label
4,0,0,6,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
130,0,0,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
198,0,0,3,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,4
253,0,0,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
282,0,0,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4


In [489]:
subcat_ind=final_cust.index
decoded_labels = label_encoder.inverse_transform(encoded_categories)
subcat=[]
for x in subcat_ind:
    subcat.append(decoded_labels[x])
print(subcat)

[' Pendants & Lockets', ' Bakeware', ' Clovia Clothing', ' Home Audio', ' HH Aviator Sunglasses']


In [493]:
row=customers.iloc[target_customer]['item_id'][-1]
df.iloc[row]['sub_category']

' Wall Decor & Clocks'